In [1]:
import pandas as pd
import numpy as np
import itertools

# 2. 시각화
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


# 3. 유틸
# tqdm 패키지는 반복문에 대해 얼마나 진척되었는지를 가시적으로 확인할 수 있도록 도와줍니다.
# https://github.com/tqdm/tqdm 사용법은 정말 간단합니다.
from tqdm.auto import tqdm


# 4. 설정
# 경고가 나와서, 출력이 많아지지 않기 위해 ignore를 설정해주었습니다.
import warnings
warnings.filterwarnings('ignore')


# 5. stats models
# 시계열 모델을 위한 ARIMA를 임포트 해주었습니다.
from statsmodels.tsa.arima_model import ARIMA
from pmdarima.arima import auto_arima

In [2]:
train_x = pd.read_csv("train_x_df.csv")
train_y = pd.read_csv("train_y_df.csv")

In [3]:
def make_mfi(my_df):
    period = 14
    result = []

    for sample_id in my_df['sample_id'].unique().tolist():
      df = my_df[my_df['sample_id'] == sample_id]

      df['typical_price'] = (df['close'] + df['high'] + df['low']) / 3
      df['money_flow'] = df['typical_price'] * df['volume']
      df['price_diff'] = df.groupby(['sample_id'])['typical_price'].diff(1).shift(-1)
      df['pf'] = np.where(df['price_diff'] > 0, df['money_flow'], 0)
      df['nf'] = np.where(df['price_diff'] < 0, df['money_flow'], 0)

      # 상승분의 14일 평균을 구해줍니다.
      # 하락분의 14일 평균을 구해줍니다.
      df["pmf"] = df["pf"].rolling(window=period, min_periods=period).sum()
      df["nmf"] = df["nf"].rolling(window=period, min_periods=period).sum()

      MFI = df['pmf'] / (df['pmf'] + df['nmf'])
      df['mfi'] = MFI
      df['mfi'] = df['mfi'].shift(1)

      result.append(df)
      
    output = pd.concat(result, axis=0)

    return output

In [4]:
train_x = make_mfi(train_x[train_x['sample_id'] < 320])
train_y = make_mfi(train_y[train_y['sample_id'] < 320])

In [5]:
TRAIN_SAMPLE_ID_LIST = train_x["sample_id"].unique().tolist()

In [6]:
def get_typical_price(df,sample_id):    
    return df[df["sample_id"] == sample_id]['typical_price'].values
def get_mfi(df,sample_id):
  return df[df["sample_id"] == sample_id]['mfi'].values

In [10]:
# AIC 값이 최소인 p,d,q 값 직접 구하기
def model_fit(df, sample_id_list):
    result = []

    for sample_id in tqdm(sample_id_list):

        price_x = get_typical_price(df, sample_id)
        mfi_series = get_mfi(df, sample_id)

        # 2. ARIMA
        # 1) 모델 정의
        ARIMA_MODEL = {}
        ARIMA_MODEL_FIT = {}

        # 2) AR 모델 적용
        try:
            ARIMA_MODEL = ARIMA(price_x, order = (4,1,1))
            ARIMA_MODEL_FIT = ARIMA_MODEL.fit(trend = 'nc', full_output = True, disp = True)

        # 3) 수렴하지 않을 경우 p d q 를 1, 1, 0으로 사용
        except:
            ARIMA_MODEL = ARIMA(price_x, order = (1,1,0))
            ARIMA_MODEL_FIT = ARIMA_MODEL.fit(trend = 'nc', full_output = True, disp = True)

        # 4) ARIMA 예측
        ARIMA_FORECAST  = ARIMA_MODEL_FIT.predict(1,120, typ='levels')

        # 3. 데이처 처리
        # 1) 최대 부분인 인덱스를 찾는데 해당 시점에 매도를 진행합니다.
        sell_time = np.argmax(ARIMA_FORECAST)

        # 2) 최대값을 찾습니다.
        max_val = np.max(ARIMA_FORECAST)

        mfi_last_val = mfi_series[1379]

        # 4. 투자 전략
        buy_quantity = 0

        # 1) typical_price가 1.15 이상이면 투자합니다.
        # if  max_val > 1.1:
        #     buy_quantity = 1

        # if mfi_last_val > 0.7:
        #     buy_quantity = 0
            
        # # 3) 만약 mfi의 값이 65 보다 크면, 초과매수 상태로 판단하여 투자하지 않습니다.
        # if mfi_last_val < 0.3  :
        #     buy_quantity = 1

        # 5. 결과
        # result_list = [
        #                 sample_id,
        #                 buy_quantity,
        #                 sell_time,
        #             ]

        # result.append(result_list)

        result_list = [
            sample_id,
            max_val,
            mfi_last_val
        ]

        # result.append(ARIMA_FORECAST)
        result.append(result_list)
    return result

In [8]:
def df2d_to_answer(df_2d):
    # valid_y_df로부터
    # open 가격 정보가 포함된
    # [샘플 수, 120분] 크기의 
    # 2차원 array를 반환하는 함수
    feature_size = df_2d.iloc[:,2:].shape[1]
    time_size = len(df_2d.time.value_counts())
    sample_size = len(df_2d.sample_id.value_counts())
    sample_index = df_2d.sample_id.value_counts().index
    array_2d = df_2d.open.values.reshape([sample_size, time_size])
    sample_index = list(sample_index)
    return array_2d, sample_index


def COIN(y_df, submission, df2d_to_answer = df2d_to_answer):
    # 2차원 데이터프레임에서 open 시점 데이터만 추출하여 array로 복원
    # sample_id정보를 index에 저장
    y_array, index = df2d_to_answer(y_df)

    
    # index 기준으로 submission을 다시 선택
    submission = submission.set_index(submission.columns[0])
    submission = submission.iloc[index, :]    
    
    # 초기 투자 비용은 10000 달러
    total_momey      = 10000 # dolors
    total_momey_list = []
    
    # 가장 처음 sample_id값
    start_index = submission.index[0]
    for row_idx in submission.index:
        sell_time  = submission.loc[row_idx, 'sell_time']
        buy_price  = y_array[row_idx - start_index, 0]
        sell_price = y_array[row_idx - start_index, sell_time]
        buy_quantity = submission.loc[row_idx, 'buy_quantity'] * total_momey
        residual = total_momey - buy_quantity
        ratio = sell_price / buy_price
        total_momey = buy_quantity * ratio * 0.9995 * 0.9995 + residual        
        total_momey_list.append(total_momey)
        
    return total_momey, total_momey_list

In [11]:
result = model_fit(train_x, TRAIN_SAMPLE_ID_LIST)
result

 33%|███▎      | 106/320 [00:48<01:51,  1.93it/s]C:\Users\user1\AppData\Local\Programs\Python\Python38\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
 37%|███▋      | 119/320 [00:54<01:41,  1.98it/s]C:\Users\user1\AppData\Local\Programs\Python\Python38\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
100%|██████████| 320/320 [02:18<00:00,  2.31it/s]


[[209, 1.3146907093333333, 0.5599256198363578],
 [164, 1.2934612258764495, 0.3784565642311226],
 [4, 1.2561274763333332, 0.4263141879703212],
 [156, 1.230245652778322, 0.454748002752641],
 [314, 1.2179777223333332, 0.35154097517748617],
 [307, 1.1932259003333334, 0.3962618304427062],
 [33, 1.1808658043333333, 0.3948632852398505],
 [141, 1.1550677613333333, 0.7693286860938006],
 [5, 1.1424722276666666, 0.46188031616758113],
 [140, 1.1294770337181668, 0.6152103495154119],
 [124, 1.1246927976666667, 0.3424012542324945],
 [168, 1.1230487823333333, 0.3093611753059728],
 [130, 1.1184693919579156, 0.5506973869713299],
 [188, 1.115092516, 0.24956476061915228],
 [153, 1.1064203583333332, 0.13529090958991105],
 [119, 1.1127778293333332, 0.26881581180170105],
 [249, 1.1122717268952942, 0.002384498957273747],
 [51, 1.107782963608479, 0.7919956349343764],
 [296, 1.1076431600069747, 0.24615248710569493],
 [212, 1.1041578361835442, 0.20576090537620606],
 [286, 1.1008676290000001, 0.3447962835873986],

In [12]:
# val_list = [1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5]
mfi_list = [0.2, 0.3, 0.4, 0.6, 0.7, 0.8]
total_money_list = []
# for val in val_list:
for mfi in mfi_list:
    my_result = []
    for i in range(len(result)):
        sell_time = np.argmax(result[i])

        # 2) 최대값을 찾습니다.
        # max_val = np.max(result[i])
        max_val = result[i][1]
        last_mfi = result[i][2]

        # 4. 투자 전략
        buy_quantity = 0

        # 1) typical_price가 1.15 이상이면 투자합니다.
        if  max_val > 1.3:
            buy_quantity = 1

        if mfi < 0.5:
            if last_mfi < mfi:
                buy_quantity = 1
        else:
            if last_mfi > mfi:
                buy_quantity = 0
        

        # 5. 결과
        result_list = [
                        i,
                        buy_quantity,
                        sell_time,
                    ]
        my_result.append(result_list)

    submit_columns = [
                "sample_id", 
                "buy_quantity", 
                "sell_time"
                ]

    submit = pd.DataFrame(data=my_result, columns=submit_columns)
    total_money, total_momey_list = COIN(train_y,
                                    submit)
    # total_money_list.append([val, total_money])
    total_money_list.append([mfi, total_money])

In [13]:
total_money_list

[[0.2, 9521.6946166163],
 [0.3, 8972.13191649372],
 [0.4, 8340.34901315803],
 [0.6, 9990.0025],
 [0.7, 9990.0025],
 [0.8, 9990.0025]]